In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os

In [2]:
df_fdc = pd.read_excel('./data/raw/Sensor_Data_DSS_반출.xlsx')

In [3]:
df_fdc.shape

(1309, 276)

동일값으로만 이루어진 열 확인

In [4]:
def find_constant_columns(df):
    constant_columns = {}
    for col in df.columns:
        if df[col].nunique() == 1:  
            constant_columns[col] = df[col].iloc[0] 
    return constant_columns

constant_columns = find_constant_columns(df_fdc)
print("같은 값으로 이루어진 열과 값:", constant_columns)

같은 값으로 이루어진 열과 값: {}


결측치 확인

In [5]:
def count_all_nan_rows(df):
    nan_rows_count = df.iloc[:, 5:].isna().all(axis=1).sum()
    print(f"모든 값이 NaN인 행의 개수: {nan_rows_count}")

count_all_nan_rows(df_fdc)

모든 값이 NaN인 행의 개수: 243


In [6]:
def remove_all_nan_rows(df):
    nan_rows = df.iloc[:, 5:].isna().all(axis=1)
    df_cleaned = df[~nan_rows]
    return df_cleaned

df_fdc_remove_missing = remove_all_nan_rows(df_fdc)
print(f'{df_fdc.shape} → {df_fdc_remove_missing.shape}')

(1309, 276) → (1066, 276)


In [7]:
count_all_nan_rows(df_fdc_remove_missing)

모든 값이 NaN인 행의 개수: 0


In [8]:
def get_columns_with_missing_values(df):
    missing_values = df.isnull().sum()  
    missing_columns = missing_values[missing_values > 0]
    return missing_columns

missing_columns = get_columns_with_missing_values(df_fdc_remove_missing)

In [9]:
def remove_columns_with_missing_values(df, missing_columns):
    df_cleaned = df.drop(columns=missing_columns.index)
    return df_cleaned

df_fdc_cleaned = remove_columns_with_missing_values(df_fdc_remove_missing, missing_columns)

In [10]:
def get_no_image_list():
    df_image = pd.read_excel('./data/raw/CO_IMAGE_DSS_반출.xlsx')
    numbers = [int(x.split('_')[1]) for x in df_image.iloc[::2].values.flatten() if isinstance(x, str) and '_' in x]
    missing_numbers = ['ID_' + str(num) for num in range(min(numbers), max(numbers) + 1) if num not in numbers]
    print("빠진 숫자:", missing_numbers)
    return missing_numbers

no_image_list = get_no_image_list()

빠진 숫자: ['ID_1231', 'ID_1233', 'ID_1239', 'ID_1241', 'ID_1244', 'ID_1245']


In [11]:
df_final = df_fdc_cleaned[~df_fdc_cleaned['ID'].isin(no_image_list)]
df_final.to_csv('./data/fdc_data.csv', index=False)

이미지 데이터 추출 및 배열 변환

In [11]:
def get_image_filenames(image_folder, image_ids):
    image_filenames = []
    for image_id in image_ids:
        image_path = os.path.join(image_folder, f"{image_id}.png")
        if os.path.exists(image_path): 
            image_filenames.append(image_path) 
        else:
            print(f"파일을 찾을 수 없습니다: {image_id}")
    return image_filenames

image_folder = './data/image'
image_ids = df_final['ID']
image_filenames = get_image_filenames(image_folder, image_ids)

In [12]:
def compare_image_order():
    image_ids = [os.path.basename(filename).split('.')[0] for filename in image_filenames]
    comparison_result = df_final['ID'].astype(str) == image_ids

    if comparison_result.all():
        print("image_filenames의 순서와 df_final['ID']의 순서가 모두 일치합니다.")
    else:
        print("일치하지 않는 항목이 있습니다.")
        mismatch_indices = comparison_result[comparison_result == False].index
        print(f"일치하지 않는 인덱스: {mismatch_indices.tolist()}")

compare_image_order()

image_filenames의 순서와 df_final['ID']의 순서가 모두 일치합니다.


In [13]:
def image_to_numpy(file_path):
    image = Image.open(file_path).convert('L')  # Convert image to grayscale
    image = image.resize((128, 128))
    return np.array(image) / 255.0

image_data = np.array([image_to_numpy(file) for file in image_filenames])

In [14]:
np.save('./data/image_data', image_data)